### 문제 2단계 체인 만들기 - 영화 추천 시스템 
문제 설명
사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그 영화의 정보(감독, 줄거리, 등장인물)들을  알려주는 2단계 체인을 구현해보세요.
요구사항
1단계 체인: 장르를 입력받아 영화 1편 추천
2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
ChatPromptTemplate 사용 
: “system”  과 “human” 메시지를 지정합니다.
두 체인을 LCEL로 연결
각 단계의 결과를 모두 출력하여 과정 확인
구현 조건
입력: 영화 장르 (예: "액션", "로맨스", "공포")
1단계 출력: 추천 영화 제목과 간단한 설명
2단계 출력: 영화 줄거리 3줄 요약


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 그냥 사람이야.") , 
     ("human", "영화 하나만 추천해줘") ]
)

# Step 1: 사용자가 입력한 재료에 따른 요리 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 추천 영화 제목 1개와 그 영화에 대한 간단한 설명")

# Step 2: 추천된 요리의 재료와 레시피 설명명
prompt2 = ChatPromptTemplate.from_template("{storyline}  영화 줄거리 3줄로 요약")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

parser = StrOutputParser()

# 장르
genre_input = "공포"

try:
    # Step 1 실행
    chain1 = prompt1 | llm | parser
    movie_info = chain1.invoke({"genre": genre_input})
    print("추천 영화:", movie_info)
    # Step 2 실행
    chain2 = prompt2 | llm | parser
    summary = chain2.invoke({"storyline": movie_info})
    print("\n줄거리 요약:\n", summary)
except Exception as e:
    print(f"오류 발생: {e}")

추천 영화: **영화:** 헤리티지 (2013)

**설명:** 가족의 오랜 전통을 계승하기 위해 조상들의 묘지에 방문한 한 가족이 겪는 기이한 현상들을 그린 영화입니다.

줄거리 요약:
 영화 '헤리티지'의 줄거리입니다.

엘리자베스는 가족의 오랜 전통을 계승하기 위해 조상들의 묘지에 방문합니다. 

그런데 그곳에서 기이한 현상들이 일어나고, 딸과 아들은 이상한 체험을 합니다.

가족은 이 기이한 현상들이 무엇인지 알 수 없어 두려움에 떨게 됩니다.
